In [39]:
import numpy as np
import pandas as pd
column_names = ['engine_id', 'cycle', 'setting1', 'setting2', 'setting3'] + [f'sensor{i}' for i in range(1, 27)]
test_df = pd.read_csv('./dataset/test_FD001.txt', sep='\s+', header=None, names=column_names)
true_rul = pd.read_csv('./dataset/RUL_FD001.txt', header=None)
test_df = test_df.dropna(axis=1, how="all")

In [40]:
import pickle

# Load the scaler from the file
with open('minmax_scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)


In [41]:
columns_to_drop = ["setting3", "sensor1", "sensor5", "sensor6", "sensor10", "sensor16", "sensor18", "sensor19"]
test_df_dropped = test_df.drop(columns=columns_to_drop)

In [42]:
columns_to_skip = test_df_dropped.columns[:2]
columns_to_normalize = test_df_dropped.columns[2:]

In [43]:
# Normalize test data using the same scaler
normalized_test_data = scaler.transform(test_df_dropped[columns_to_normalize])

# Combine normalized and unnormalized columns in test data
test_df_normalized = pd.DataFrame(test_df_dropped[columns_to_skip].values, columns=columns_to_skip)
test_df_normalized = pd.concat([test_df_normalized, pd.DataFrame(normalized_test_data, columns=columns_to_normalize)], axis=1)

In [44]:
from tensorflow.keras.models import load_model

# Load the model from the HDF5 file
model = load_model('pwrul_prediction_model.h5')

In [45]:
from sklearn.metrics import mean_squared_error

# Identify feature columns
feature_columns = [col for col in test_df_normalized.columns if col not in ['engine_id', 'cycle', 'RUL', 'PWRUL']]

# Define sequence length
sequence_length = 30

X_test = []
y_test = []

for engine_id in test_df_normalized['engine_id'].unique():
    engine_data = test_df_normalized[test_df_normalized['engine_id'] == engine_id].reset_index(drop=True)
    if len(engine_data) >= sequence_length:
        # Use only the last sequence
        seq_x = engine_data[feature_columns].iloc[-sequence_length:].values
        X_test.append(seq_x)
        # Get the true RUL for this engine
        seq_y = true_rul.loc[engine_id - 1].values[0]
        y_test.append(seq_y)
    else:
        print(f"Engine {engine_id} has insufficient data for the defined sequence length.")

X_test = np.array(X_test)
y_test = np.array(y_test)

# Predict RUL on test data
y_test_pred = model.predict(X_test)

# Evaluate
test_mse = mean_squared_error(y_test, y_test_pred)
print("Test Mean Squared Error:", test_mse)

4/4 [==============================] - 0s 2ms/step
Test Mean Squared Error: 222.4735336860263
